# Benchmarking Aura features

In [1]:
import timeit
from graphdatascience.aura_graph_data_science import AuraGraphDataScience
from graphdatascience.aura_sessions import AuraSessions
from graphdatascience.query_runner.aura_db_arrow_query_runner import AuraDbConnectionInfo

In [2]:
CLIENT_ID = os.environ["CLIENT_ID"]
CLIENT_SECRET = os.environ["CLIENT_SECRET"]
DB_PASSWORD = "" 
DB_ID = "b1c3673e"
os.environ["AURA_ENV"] = "devstrawberry"

db_connection_info = AuraDbConnectionInfo(
        f"neo4j+s://{DB_ID}-{os.environ['AURA_ENV']}.databases.neo4j-dev.io", ("neo4j", DB_PASSWORD)
    )

In [3]:
sessions = AuraSessions(db_connection_info, (CLIENT_ID, CLIENT_SECRET))
session_name = "benchmark-session"
session_pw = "my-password"

print("Starting GDS session")
if session_name in [i.name for i in sessions.list_sessions()]:
    gds = sessions.connect(session_name, session_pw)
else:
    gds = sessions.create_gds(session_name, session_pw)

Starting GDS session


Unable to retrieve routing information
Unable to connect to the Neo4j DBMS. Trying again...
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to connect to the Neo4j DBMS. Trying again...
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to retrieve routing information
Unable to connect to the Neo4j DBMS. Trying again...
Unable to retrieve rou

In [5]:
def workload_remote_projection():  
  with gds.graph.project.remoteDb(
        "graph",
        """
    CYPHER runtime = parallel
    MATCH (u)
    OPTIONAL MATCH (u)-[r]->(t)
    RETURN gds.graph.project.remote(u, t, {})
    """,
    concurrency=4
    ) as G:
        print(G)

In [7]:
iterations = 6
result = timeit.repeat(lambda: workload_remote_projection(), number=1, repeat=iterations)

print(result)

Graph(name=graph, node_count=1000, relationship_count=8000)
Graph(name=graph, node_count=1000, relationship_count=8000)
Graph(name=graph, node_count=1000, relationship_count=8000)
Graph(name=graph, node_count=1000, relationship_count=8000)
Graph(name=graph, node_count=1000, relationship_count=8000)
Graph(name=graph, node_count=1000, relationship_count=8000)
[4.892199249996338, 2.424036000011256, 2.5078814169974066, 2.2833090419881046, 2.2362234580214135, 2.2035216249933]


## Write back

In [16]:
G, _ = gds.graph.project.remoteDb(
        "graph",
        """
    CYPHER runtime = parallel
    MATCH (u)
    OPTIONAL MATCH (u)-[r]->(t)
    RETURN gds.graph.project.remote(u, t, {})
    """,
    concurrency=4
    )

gds.degree.mutate(G, mutateProperty="degree")
gds.fastRP.mutate(G, mutateProperty="embedding", embeddingDimension=128, iterationWeights=[1.0])
gds.knn.mutate(G, topK=4, mutateRelationshipType="KNN_RELS", mutateProperty="score", maxIterations=1, nodeProperties=["embedding"])

ranIterations                                                             1
nodePairsConsidered                                                   22170
didConverge                                                           False
preProcessingMillis                                                       0
computeMillis                                                            67
mutateMillis                                                             65
postProcessingMillis                                                      0
nodesCompared                                                          1000
relationshipsWritten                                                   4000
similarityDistribution    {'min': 0.0, 'p5': 0.5340614318847656, 'max': ...
configuration             {'mutateProperty': 'score', 'jobId': '6f156d35...
Name: 0, dtype: object

In [20]:
result = timeit.repeat(
    lambda: gds.run_cypher("MATCH (n) SET n.degree = null"),
    lambda: gds.graph.nodeProperties.write(G, node_properties=["degree"]), 
    number=1, 
    repeat=iterations)

print(f"scalar property: {result}")

scalar property: [0.12835391599219292, 0.12202195799909532, 0.12279637498431839, 0.12687449998338707, 0.137083875015378, 0.14937504200497642]


In [19]:
result = timeit.repeat(
    lambda: gds.run_cypher("MATCH (n) SET n.embedding = null"),
    lambda: gds.graph.nodeProperties.write(G, node_properties=["embedding"]), 
    number=1, 
    repeat=iterations)

print(f"array property: {result}")

array property: [0.19451499998103827, 0.1829530840041116, 0.18628650001483038, 0.17463279099320062, 0.20477325000683777, 0.1867185410228558]


In [24]:
result = timeit.repeat(
    lambda: gds.run_cypher("MATCH (n)-[r:KNN_RELS]->() DELETE r"),
    lambda: gds.graph.relationship.write(G, relationship_type="KNN_RELS", relationship_property="score"), 
    number=1, 
    repeat=iterations)

print(f"relationships: {result}")

relationships: [0.21328462500241585, 0.19357400000444613, 0.1947907089779619, 0.20090529101435095, 0.20773408300010487, 0.18440237498725764]


## Cleanup

In [15]:
G.drop()

graphName                                                            graph
database                                                             neo4j
databaseLocation                                                    remote
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                             1000
relationshipCount                                                     8000
configuration                                                         None
density                                                           0.008008
creationTime                           2023-11-27T15:30:57.317400552+00:00
modificationTime                       2023-11-27T15:30:59.429387718+00:00
schema                   {'graphProperties': {}, 'nodes': {'__ALL__': {...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'__ALL__': {...
Name: 0, dtype: object

In [ ]:
sessions.delete_gds(session_name)

In [ ]:
# cleanup

from graphdatascience.aura_api import AuraApi

aura_api = AuraApi(CLIENT_ID, CLIENT_SECRET)
aura_api.delete_instance(DB_ID)